In [1]:
# Install condacolab
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:08
🔁 Restarting kernel...


In [1]:
# Create a Python 3.7 environment
!conda create -n scene_graph_env python=3.7 -y

# Install PyTorch 1.4.0 and torchvision 0.5.0 using pip
!conda run -n scene_graph_env pip install torch==1.4.0 torchvision==0.5.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html

# Verify the installation
!conda run -n scene_graph_env python -c "import torch; print(f'PyTorch version: {torch.__version__}'); print(f'CUDA available: {torch.cuda.is_available()}'); import torchvision; print(f'torchvision version: {torchvision.__version__}')"

Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.2
    latest version: 25.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/scene_graph_env

  added / updated specs:
    - python=3.7


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.4.26  |       hbd8a1cb_0         149 KB  conda-forge
    ld_impl_linux-64-2.43      |       h712a8e2_4         656 KB  conda-forge
    libffi-3.4.6               |       h2dba641_1          56 KB  conda-forge
    libgcc-15.1.0              |       h767d61c_2         810 KB  conda-forge
    libgcc-ng-15.1.0           |       h69a702a_2          34 KB  conda-forge
    libgomp-15.1.0             |       h767d61c_2         442 KB  conda-for

In [2]:
# Install other required packages
!conda run -n scene_graph_env pip install ninja yacs cython matplotlib tqdm opencv-python overrides scipy h5py ipython gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.8/793.8 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1

In [3]:
# Remove all existing directories
!rm -rf cocoapi apex Scene

# Verify they're all gone
!ls -la | grep -E "(cocoapi|apex|Scene)"

In [4]:
%%bash
# Clone required repositories
git clone https://github.com/cocodataset/cocoapi.git
git clone https://github.com/NVIDIA/apex.git
git clone https://github.com/KaihuaTang/Scene-Graph-Benchmark.pytorch.git Scene

# Install COCO API
cd cocoapi/PythonAPI
conda run -n scene_graph_env python setup.py build_ext install
cd ../..

# Install apex with specific commit compatible with PyTorch 1.4
cd apex
git reset --hard 3fe10b5597ba14a748ebb271a6ab97c09c5701ac
conda run -n scene_graph_env pip install -v --no-cache-dir --global-option="--cpp_ext" ./
cd ..

# Build and install Scene Graph Benchmark
cd Scene
conda run -n scene_graph_env python setup.py build develop
cd ..

# Create directories for checkpoints and data
mkdir -p Scene/checkpoint/pretrained_faster_rcnn/
mkdir -p Scene/datasets/vg
mkdir -p Scene/glove

running build_ext
Compiling pycocotools/_mask.pyx because it changed.
[1/1] Cythonizing pycocotools/_mask.pyx
building 'pycocotools._mask' extension
creating build
creating build/common
creating build/temp.linux-x86_64-cpython-37
creating build/temp.linux-x86_64-cpython-37/pycocotools
gcc -pthread -B /usr/local/envs/scene_graph_env/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/usr/local/envs/scene_graph_env/lib/python3.7/site-packages/numpy/core/include -I../common -I/usr/local/envs/scene_graph_env/include/python3.7m -c ../common/maskApi.c -o build/temp.linux-x86_64-cpython-37/../common/maskApi.o -Wno-cpp -Wno-unused-function -std=c99
gcc -pthread -B /usr/local/envs/scene_graph_env/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/usr/local/envs/scene_graph_env/lib/python3.7/site-packages/numpy/core/include -I../common -I/usr/local/envs/scene_graph_env/include/python3.7m 

Cloning into 'cocoapi'...
Cloning into 'apex'...
Cloning into 'Scene'...
../common/maskApi.c: In function ‘rleDecode’:
../common/maskApi.c:46:7: warning: this ‘for’ clause does not guard... [-Wmisleading-indentation]
   46 |       for( k=0; k<R[i].cnts[j]; k++ ) *(M++)=v; v=!v; }}
      |       ^~~
../common/maskApi.c:46:49: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the ‘for’
   46 |       for( k=0; k<R[i].cnts[j]; k++ ) *(M++)=v; v=!v; }}
      |                                                 ^
../common/maskApi.c: In function ‘rleFrPoly’:
../common/maskApi.c:166:3: warning: this ‘for’ clause does not guard... [-Wmisleading-indentation]
  166 |   for(j=0; j<k; j++) x[j]=(int)(scale*xy[j*2+0]+.5); x[k]=x[0];
      |   ^~~
../common/maskApi.c:166:54: note: ...this statement, but the latter is misleadingly indented as if it were guarded by the ‘for’
  166 |   for(j=0; j<k; j++) x[j]=(int)(scale*xy[j*2+0]+.5); x[k]=x[0];
      |            

In [5]:
# Fix the base DATA_DIR (this is all you need!)
!sed -i 's|DATA_DIR = "/media/rafi/Samsung_T5/_DATASETS/"|DATA_DIR = "/content/Scene/datasets/"|g' Scene/maskrcnn_benchmark/config/paths_catalog.py

# Verify the fix
!grep -n "DATA_DIR" Scene/maskrcnn_benchmark/config/paths_catalog.py
!grep -n "/media/rafi/" Scene/maskrcnn_benchmark/config/paths_catalog.py || echo "No more hardcoded paths found"

9:    #DATA_DIR = "/home/users/alatif/data/ImageCorpora/"
10:    DATA_DIR = "/content/Scene/datasets/"
127:            data_dir = DatasetCatalog.DATA_DIR
138:            data_dir = DatasetCatalog.DATA_DIR
153:            data_dir = DatasetCatalog.DATA_DIR
No more hardcoded paths found


In [18]:
# Let's check what's actually in your Google Drive
print("=== CHECKING GOOGLE DRIVE MOUNT ===")

# Check if Google Drive is mounted
import os
if os.path.exists('/content/drive'):
    print("✓ Google Drive is mounted")
else:
    print("✗ Google Drive not mounted - let's remount")
    from google.colab import drive
    drive.mount('/content/drive')

# # List the contents of MyDrive
# print("\n=== CONTENTS OF MyDrive ===")
# !ls -la /content/drive/MyDrive/

# # Look for folders with similar names
# print("\n=== SEARCHING FOR FOLDERS WITH 'upload' or 'sgdet' ===")
# !find /content/drive/MyDrive/ -type d -name "*upload*" 2>/dev/null
# !find /content/drive/MyDrive/ -type d -name "*sgdet*" 2>/dev/null
# !find /content/drive/MyDrive/ -type d -name "*causal*" 2>/dev/null
# !find /content/drive/MyDrive/ -type d -name "*motif*" 2>/dev/null

=== CHECKING GOOGLE DRIVE MOUNT ===
✓ Google Drive is mounted


In [19]:
# Method 1: Direct upload of the dictionary file
from google.colab import files
print("Please upload the VG-SGG-dicts-with-attri.json file:")
uploaded = files.upload()  # This will show a file picker dialog

# Move to the correct location
!mv VG-SGG-dicts-with-attri.json Scene/datasets/vg/

Please upload the VG-SGG-dicts-with-attri.json file:


Saving VG-SGG-dicts-with-attri.json to VG-SGG-dicts-with-attri.json


In [ ]:

from google.colab import drive
import os

# UPDATE THIS PATH to where you uploaded your files in Google Drive
drive_folder_path = "/content/drive/MyDrive/upload_causal_motif_sgdet"
print(f"Looking for files in: {drive_folder_path}")

# Check if the folder exists
if os.path.exists(drive_folder_path):
    print("✓ Folder found! Files in the folder:")
    !ls -lh "{drive_folder_path}"

    # Define file mappings
    model_files = [
        'model_0028000.pth',
        'last_checkpoint',
        'labels.json',
        'config.yml',
        'log.txt',
        'VG_stanford_filtered_with_attribute_train_statistics.cache'
    ]

    # Dataset files that go to Scene/datasets/vg/
    dataset_files = [
        'VG-SGG-dicts-with-attri.json',
        'VG-SGG-with-attri.h5',       # The big one you need to upload
        'image_data.json'              # Also needed (smaller file)
    ]

    print("\n=== COPYING MODEL FILES ===")
    for file_name in model_files:
        source = f"{drive_folder_path}/{file_name}"
        dest = f"Scene/checkpoint/pretrained_faster_rcnn/{file_name}"

        if os.path.exists(source):
            print(f"Copying {file_name}... (this may take a while for large files)")
            !cp "{source}" "{dest}"

            # Check if copy was successful
            if os.path.exists(dest):
                file_size = os.path.getsize(dest)
                print(f"✓ {file_name} copied successfully ({file_size:,} bytes)")
            else:
                print(f"✗ Failed to copy {file_name}")
        else:
            print(f"✗ File not found: {source}")

    print("\n=== COPYING DATASET FILES ===")
    for file_name in dataset_files:
        source = f"{drive_folder_path}/{file_name}"
        dest = f"Scene/datasets/vg/{file_name}"

        if os.path.exists(source):
            if file_name.endswith('.h5'):
                print(f"Copying {file_name}... (this is a large file, may take several minutes)")
            else:
                print(f"Copying {file_name}...")
            !cp "{source}" "{dest}"

            if os.path.exists(dest):
                file_size = os.path.getsize(dest)
                print(f"✓ {file_name} copied successfully ({file_size:,} bytes)")
            else:
                print(f"✗ Failed to copy {file_name}")
        else:
            print(f"✗ File not found: {source}")
            if file_name == 'VG-SGG-with-attri.h5':
                print("   This is the main dataset file - you need to upload it to Google Drive")
            elif file_name == 'image_data.json':
                print("   This file is also required. You may need to download it separately")

else:
    print(f"✗ Folder not found: {drive_folder_path}")
    print("Please create this folder in Google Drive and upload your files there.")
    print("Or update the path above to match where you uploaded the files.")

print("\n=== REQUIRED FILES CHECKLIST ===")
required_files = {
    'Scene/checkpoint/pretrained_faster_rcnn/model_0028000.pth': 'Model checkpoint',
    'Scene/datasets/vg/VG-SGG-dicts-with-attri.json': 'Object/predicate dictionary',
    'Scene/datasets/vg/VG-SGG-with-attri.h5': 'Main dataset file (large)',
    'Scene/datasets/vg/image_data.json': 'Image metadata'
}

print("Checking for all required files:")
all_present = True
for file_path, description in required_files.items():
    if os.path.exists(file_path):
        file_size = os.path.getsize(file_path)
        print(f"✓ {description}: {file_size:,} bytes")
    else:
        print(f"✗ {description}: MISSING")
        all_present = False

if all_present:
    print("\n🎉 All required files are present! You should be able to run the model now.")
else:
    print("\n❌ Some files are missing. Upload them to Google Drive first.")


Looking for files in: /content/drive/MyDrive/upload_causal_motif_sgdet
✓ Folder found! Files in the folder:
total 2.2G
-rw------- 1 root root 7.9K Mar 11  2020 config.yml
-rw------- 1 root root  21M May 13 11:05 image_data.json
-rw------- 1 root root 2.6K Mar 11  2020 labels.json
-rw------- 1 root root   68 Mar 11  2020 last_checkpoint
-rw------- 1 root root 198K Mar 11  2020 log.txt
-rw------- 1 root root 2.2G Mar 11  2020 model_0028000.pth
-rw------- 1 root root  14M Mar 11  2020 VG_stanford_filtered_with_attribute_train_statistics.cache

=== COPYING MODEL FILES ===
Copying model_0028000.pth... (this may take a while for large files)


In [11]:
# Mount Google Drive to access your images
from google.colab import drive
drive.mount('/content/drive')

# Create a folder to store output
!mkdir -p /content/Scene/output/scene_graphs

# Specify the path to your referring expression images
referring_expr_path = "/content/drive/MyDrive/Gaala-pics"  # Change this to your actual path
print(f"Using image path: {referring_expr_path}")

# Check if the directory exists and has images
!ls -la "{referring_expr_path}"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using image path: /content/drive/MyDrive/Gaala-pics
total 21098
-rw------- 1 root root 2888091 May 13 09:13 DSC_8965.JPG
-rw------- 1 root root 2747610 May 13 09:13 DSC_8966.JPG
-rw------- 1 root root 2360925 May 13 09:12 DSC_8972.JPG
-rw------- 1 root root 1444408 May 13 09:10 DSC_9348.JPG
-rw------- 1 root root 2330892 May 13 09:10 DSC_9418.JPG
-rw------- 1 root root 1313187 May 13 09:09 DSC_9471.JPG
-rw------- 1 root root 2149365 May 13 09:09 DSC_9474.JPG
-rw------- 1 root root 2806638 May 13 09:08 DSC_9727.JPG
-rw------- 1 root root 2115533 May 13 09:08 DSC_9774.JPG
-rw------- 1 root root 1445684 May 13 09:08 DSC_9777.JPG


In [14]:
import h5py
import json
import numpy as np
import os

print("=== CREATING MINIMAL DUMMY DATASET FILES ===")

# Create minimal dummy VG-SGG-with-attri.h5 file
print("Creating dummy VG-SGG-with-attri.h5...")
try:
    with h5py.File('Scene/datasets/vg/VG-SGG-with-attri.h5', 'w') as f:
        # Add minimal required datasets based on the error trace
        f.create_dataset('split', data=np.array([0, 1, 2]))
        f.create_dataset('labels', data=np.array([[1, 2, 3]]))
        f.create_dataset('relationships', data=np.array([[0, 1, 1]]))
        f.create_dataset('img_to_first_rel', data=np.array([0, 1]))
        f.create_dataset('img_to_last_rel', data=np.array([0, 1]))
        f.create_dataset('img_to_first_box', data=np.array([0, 1]))
        f.create_dataset('img_to_last_box', data=np.array([0, 1]))
        f.create_dataset('boxes_1024', data=np.array([[0, 0, 100, 100]]))
        f.create_dataset('boxes_512', data=np.array([[0, 0, 50, 50]]))
        f.create_dataset('active_object_mask', data=np.array([1]))
        f.create_dataset('predicates', data=np.array([1]))
        f.create_dataset('objects', data=np.array([1]))
        f.create_dataset('attributes', data=np.array([1]))
    print("✓ Dummy VG-SGG-with-attri.h5 created")
except Exception as e:
    print(f"✗ Error creating H5 file: {e}")

# Create minimal image_data.json
print("Creating dummy image_data.json...")
try:
    dummy_image_data = [{"image_id": 1, "width": 1024, "height": 768, "url": "dummy.jpg"}]
    with open('Scene/datasets/vg/image_data.json', 'w') as f:
        json.dump(dummy_image_data, f)
    print("✓ Dummy image_data.json created")
except Exception as e:
    print(f"✗ Error creating image_data.json: {e}")

# Create dummy vg_capgraphs_anno.json
print("Creating dummy vg_capgraphs_anno.json...")
try:
    dummy_capgraphs = {"1": {"objects": [], "relationships": []}}
    with open('Scene/datasets/vg/vg_capgraphs_anno.json', 'w') as f:
        json.dump(dummy_capgraphs, f)
    print("✓ Dummy vg_capgraphs_anno.json created")
except Exception as e:
    print(f"✗ Error creating vg_capgraphs_anno.json: {e}")

# Verify the files exist
print("\n=== VERIFICATION ===")
required_files = [
    'Scene/datasets/vg/VG-SGG-with-attri.h5',
    'Scene/datasets/vg/image_data.json',
    'Scene/datasets/vg/vg_capgraphs_anno.json',
    'Scene/datasets/vg/VG-SGG-dicts-with-attri.json'
]

for file_path in required_files:
    if os.path.exists(file_path):
        size = os.path.getsize(file_path)
        print(f"✓ {file_path} exists ({size:,} bytes)")
    else:
        print(f"✗ {file_path} missing")

print("\n=== READY TO RUN MODEL ===")

=== CREATING MINIMAL DUMMY DATASET FILES ===
Creating dummy VG-SGG-with-attri.h5...
✓ Dummy VG-SGG-with-attri.h5 created
Creating dummy image_data.json...
✓ Dummy image_data.json created
Creating dummy vg_capgraphs_anno.json...
✓ Dummy vg_capgraphs_anno.json created

=== VERIFICATION ===
✓ Scene/datasets/vg/VG-SGG-with-attri.h5 exists (7,672 bytes)
✓ Scene/datasets/vg/image_data.json exists (67 bytes)
✓ Scene/datasets/vg/vg_capgraphs_anno.json exists (43 bytes)
✓ Scene/datasets/vg/VG-SGG-dicts-with-attri.json exists (18,482 bytes)

=== READY TO RUN MODEL ===


In [15]:
%%bash
# Run inference with the Causal TDE model on your images
cd Scene
conda run -n scene_graph_env python -m torch.distributed.launch --master_port 10027 --nproc_per_node=1 tools/relation_test_net.py \
--config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" \
MODEL.ROI_RELATION_HEAD.USE_GT_BOX False \
MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False \
MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor \
MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE TDE \
MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum \
MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs \
TEST.IMS_PER_BATCH 1 \
DTYPE "float16" \
GLOVE_DIR /content/Scene/glove \
MODEL.PRETRAINED_DETECTOR_CKPT /content/Scene/checkpoint/pretrained_faster_rcnn/model_0028000.pth \
OUTPUT_DIR /content/Scene/output \
TEST.CUSTUM_EVAL True \
TEST.CUSTUM_PATH "/content/drive/MyDrive/Gaala-pics" \
DETECTED_SGG_DIR /content/Scene/output/scene_graphs

2025-05-28 13:39:09,997 maskrcnn_benchmark INFO: Using 1 GPUs
2025-05-28 13:39:09,997 maskrcnn_benchmark INFO: AMP_VERBOSE: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  NUM_WORKERS: 4
  SIZE_DIVISIBILITY: 32
DATASETS:
  TEST: ('VG_stanford_filtered_with_attribute_test',)
  TO_TEST: None
  TRAIN: ('VG_stanford_filtered_with_attribute_train',)
  VAL: ('VG_stanford_filtered_with_attribute_val',)
DETECTED_SGG_DIR: /content/Scene/output/scene_graphs
DTYPE: float16
GLOVE_DIR: /content/Scene/glove
INPUT:
  BRIGHTNESS: 0.0
  CONTRAST: 0.0
  HUE: 0.0
  MAX_SIZE_TEST: 1000
  MAX_SIZE_TRAIN: 1000
  MIN_SIZE_TEST: 600
  MIN_SIZE_TRAIN: (600,)
  PIXEL_MEAN: [102.9801, 115.9465, 122.7717]
  PIXEL_STD: [1.0, 1.0, 1.0]
  SATURATION: 0.0
  TO_BGR255: True
  VERTICAL_FLIP_PROB_TRAIN: 0.0
MODEL:
  ATTRIBUTE_ON: False
  BACKBONE:
    CONV_BODY: R-101-FPN
    FREEZE_CONV_BODY_AT: 2
  CLS_AGNOSTIC_BBOX_REG: False
  DEVICE: cuda
  FBNET:
    ARCH: default
    ARCH_DEF: 
    BN_TYPE: bn
    DET_HEAD_BLOC


100%|██████████| 10/10 [00:01<00:00,  7.98it/s]
Traceback (most recent call last):
  File "tools/relation_test_net.py", line 123, in <module>
    main()
  File "tools/relation_test_net.py", line 67, in main
    model = build_detection_model(cfg)
  File "/content/Scene/maskrcnn_benchmark/modeling/detector/detectors.py", line 10, in build_detection_model
    return meta_arch(cfg)
  File "/content/Scene/maskrcnn_benchmark/modeling/detector/generalized_rcnn.py", line 31, in __init__
    self.roi_heads = build_roi_heads(cfg, self.backbone.out_channels)
  File "/content/Scene/maskrcnn_benchmark/modeling/roi_heads/roi_heads.py", line 89, in build_roi_heads
    roi_heads.append(("relation", build_roi_relation_head(cfg, in_channels)))
  File "/content/Scene/maskrcnn_benchmark/modeling/roi_heads/relation_head/relation_head.py", line 105, in build_roi_relation_head
    return ROIRelationHead(cfg, in_channels)
  File "/content/Scene/maskrcnn_benchmark/modeling/roi_heads/relation_head/relation_hea

CalledProcessError: Command 'b'# Run inference with the Causal TDE model on your images\ncd Scene\nconda run -n scene_graph_env python -m torch.distributed.launch --master_port 10027 --nproc_per_node=1 tools/relation_test_net.py \\\n--config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" \\\nMODEL.ROI_RELATION_HEAD.USE_GT_BOX False \\\nMODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False \\\nMODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor \\\nMODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE TDE \\\nMODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum \\\nMODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs \\\nTEST.IMS_PER_BATCH 1 \\\nDTYPE "float16" \\\nGLOVE_DIR /content/Scene/glove \\\nMODEL.PRETRAINED_DETECTOR_CKPT /content/Scene/checkpoint/pretrained_faster_rcnn/model_0028000.pth \\\nOUTPUT_DIR /content/Scene/output \\\nTEST.CUSTUM_EVAL True \\\nTEST.CUSTUM_PATH "/content/drive/MyDrive/Gaala-pics" \\\nDETECTED_SGG_DIR /content/Scene/output/scene_graphs\n'' returned non-zero exit sta

In [13]:
%%bash
cd Scene
conda run -n scene_graph_env python -m torch.distributed.launch --master_port 10027 --nproc_per_node=1 tools/relation_test_net.py \
--config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" \
MODEL.ROI_RELATION_HEAD.USE_GT_BOX False \
MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False \
MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor \
MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE TDE \
MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum \
MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs \
TEST.IMS_PER_BATCH 1 \
DTYPE "float16" \
GLOVE_DIR /content/Scene/glove \
MODEL.PRETRAINED_DETECTOR_CKPT /content/Scene/checkpoint/pretrained_faster_rcnn/model_0028000.pth \
OUTPUT_DIR /content/Scene/output \
TEST.CUSTUM_EVAL True \
TEST.CUSTUM_PATH "/content/drive/MyDrive/Gaala-pics" \
DETECTED_SGG_DIR /content/Scene/output/scene_graphs \
MODEL.ROI_RELATION_HEAD.USE_DATASET_STATISTICS False

Traceback (most recent call last):
  File "tools/relation_test_net.py", line 123, in <module>
    main()
  File "tools/relation_test_net.py", line 56, in main
    cfg.merge_from_list(args.opts)
  File "/usr/local/envs/scene_graph_env/lib/python3.7/site-packages/yacs/config.py", line 243, in merge_from_list
    _assert_with_logging(subkey in d, "Non-existent key: {}".format(full_key))
  File "/usr/local/envs/scene_graph_env/lib/python3.7/site-packages/yacs/config.py", line 545, in _assert_with_logging
    assert cond, msg
AssertionError: Non-existent key: MODEL.ROI_RELATION_HEAD.USE_DATASET_STATISTICS
Traceback (most recent call last):
  File "/usr/local/envs/scene_graph_env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/envs/scene_graph_env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/envs/scene_graph_env/lib/python3.7/site-packages/torch/distributed/launch.py", line 263, in <module

CalledProcessError: Command 'b'cd Scene\nconda run -n scene_graph_env python -m torch.distributed.launch --master_port 10027 --nproc_per_node=1 tools/relation_test_net.py \\\n--config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" \\\nMODEL.ROI_RELATION_HEAD.USE_GT_BOX False \\\nMODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False \\\nMODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor \\\nMODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE TDE \\\nMODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum \\\nMODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs \\\nTEST.IMS_PER_BATCH 1 \\\nDTYPE "float16" \\\nGLOVE_DIR /content/Scene/glove \\\nMODEL.PRETRAINED_DETECTOR_CKPT /content/Scene/checkpoint/pretrained_faster_rcnn/model_0028000.pth \\\nOUTPUT_DIR /content/Scene/output \\\nTEST.CUSTUM_EVAL True \\\nTEST.CUSTUM_PATH "/content/drive/MyDrive/Gaala-pics" \\\nDETECTED_SGG_DIR /content/Scene/output/scene_graphs \\\nMODEL.ROI_RELATION_HEAD.USE_DATASET_STATISTICS False\n'' returned non-zero exit stat

In [16]:
%%bash
cd Scene
conda run -n scene_graph_env python -m torch.distributed.launch --master_port 10027 --nproc_per_node=1 tools/relation_test_net.py \
--config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" \
MODEL.ROI_RELATION_HEAD.USE_GT_BOX False \
MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False \
MODEL.ROI_RELATION_HEAD.PREDICTOR MotifPredictor \
TEST.IMS_PER_BATCH 1 \
DTYPE "float16" \
GLOVE_DIR /content/Scene/glove \
MODEL.PRETRAINED_DETECTOR_CKPT /content/Scene/checkpoint/pretrained_faster_rcnn/model_0028000.pth \
OUTPUT_DIR /content/Scene/output \
TEST.CUSTUM_EVAL True \
TEST.CUSTUM_PATH "/content/drive/MyDrive/Gaala-pics" \
DETECTED_SGG_DIR /content/Scene/output/scene_graphs


2025-05-28 13:42:51,633 maskrcnn_benchmark INFO: Using 1 GPUs
2025-05-28 13:42:51,633 maskrcnn_benchmark INFO: AMP_VERBOSE: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  NUM_WORKERS: 4
  SIZE_DIVISIBILITY: 32
DATASETS:
  TEST: ('VG_stanford_filtered_with_attribute_test',)
  TO_TEST: None
  TRAIN: ('VG_stanford_filtered_with_attribute_train',)
  VAL: ('VG_stanford_filtered_with_attribute_val',)
DETECTED_SGG_DIR: /content/Scene/output/scene_graphs
DTYPE: float16
GLOVE_DIR: /content/Scene/glove
INPUT:
  BRIGHTNESS: 0.0
  CONTRAST: 0.0
  HUE: 0.0
  MAX_SIZE_TEST: 1000
  MAX_SIZE_TRAIN: 1000
  MIN_SIZE_TEST: 600
  MIN_SIZE_TRAIN: (600,)
  PIXEL_MEAN: [102.9801, 115.9465, 122.7717]
  PIXEL_STD: [1.0, 1.0, 1.0]
  SATURATION: 0.0
  TO_BGR255: True
  VERTICAL_FLIP_PROB_TRAIN: 0.0
MODEL:
  ATTRIBUTE_ON: False
  BACKBONE:
    CONV_BODY: R-101-FPN
    FREEZE_CONV_BODY_AT: 2
  CLS_AGNOSTIC_BBOX_REG: False
  DEVICE: cuda
  FBNET:
    ARCH: default
    ARCH_DEF: 
    BN_TYPE: bn
    DET_HEAD_BLOC


100%|██████████| 10/10 [00:01<00:00,  8.33it/s]
Traceback (most recent call last):
  File "tools/relation_test_net.py", line 123, in <module>
    main()
  File "tools/relation_test_net.py", line 67, in main
    model = build_detection_model(cfg)
  File "/content/Scene/maskrcnn_benchmark/modeling/detector/detectors.py", line 10, in build_detection_model
    return meta_arch(cfg)
  File "/content/Scene/maskrcnn_benchmark/modeling/detector/generalized_rcnn.py", line 31, in __init__
    self.roi_heads = build_roi_heads(cfg, self.backbone.out_channels)
  File "/content/Scene/maskrcnn_benchmark/modeling/roi_heads/roi_heads.py", line 89, in build_roi_heads
    roi_heads.append(("relation", build_roi_relation_head(cfg, in_channels)))
  File "/content/Scene/maskrcnn_benchmark/modeling/roi_heads/relation_head/relation_head.py", line 105, in build_roi_relation_head
    return ROIRelationHead(cfg, in_channels)
  File "/content/Scene/maskrcnn_benchmark/modeling/roi_heads/relation_head/relation_hea

CalledProcessError: Command 'b'cd Scene\nconda run -n scene_graph_env python -m torch.distributed.launch --master_port 10027 --nproc_per_node=1 tools/relation_test_net.py \\\n--config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" \\\nMODEL.ROI_RELATION_HEAD.USE_GT_BOX False \\\nMODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False \\\nMODEL.ROI_RELATION_HEAD.PREDICTOR MotifPredictor \\\nTEST.IMS_PER_BATCH 1 \\\nDTYPE "float16" \\\nGLOVE_DIR /content/Scene/glove \\\nMODEL.PRETRAINED_DETECTOR_CKPT /content/Scene/checkpoint/pretrained_faster_rcnn/model_0028000.pth \\\nOUTPUT_DIR /content/Scene/output \\\nTEST.CUSTUM_EVAL True \\\nTEST.CUSTUM_PATH "/content/drive/MyDrive/Gaala-pics" \\\nDETECTED_SGG_DIR /content/Scene/output/scene_graphs\n'' returned non-zero exit status 1.

# hardware and OS
To run this repo, you need 1 to 8 GPUs with CUDA. If you don't have a Nvidia Graphic card, no problem! This notebook is intended to run on Google Colab!

Don't forget to activate the GPU in **Goolgle Colab** ([how to](https://jovianlin.io/pytorch-with-gpu-in-google-colab/)) If you do not enable GPU from the beggining then you will have to restart te setup from beggining

If using **Kaggle kernel** do not forget to enable internet

**OS:** It is almost impossible to run this repo on Windows (believe me, I tried hard) because of many dependencies that are made for Linux. So better continue with Linux.

# Environment setup
before installing anything related to this project, first you need a version of PyTorch compatible with your GPU settings

with the 2 line script below you can check if your environment is ok.

In [ ]:
! wget --quiet "https://raw.githubusercontent.com/pytorch/pytorch/master/torch/utils/collect_env.py"
! python collect_env.py

PyTorch version: 1.5.0+cu101
Is debug build: No
CUDA used to build PyTorch: 10.1

OS: Ubuntu 16.04.4 LTS
GCC version: (Ubuntu 5.4.0-6ubuntu1~16.04.9) 5.4.0 20160609
CMake version: Could not collect

Python version: 3.7
Is CUDA available: Yes
CUDA runtime version: 10.1.105
GPU models and configuration: 
GPU 0: Tesla K80
GPU 1: Tesla K80

Nvidia driver version: 430.26
cuDNN version: /usr/lib/x86_64-linux-gnu/libcudnn.so.7.6.5

Versions of relevant libraries:
[pip3] numpy==1.18.4
[pip3] torch==1.5.0+cu101
[pip3] torchvision==0.6.0+cu101
[conda] mkl                       2018.0.1             h19d6760_4  
[conda] mkl-service               1.1.2            py36h17a0993_4  
[conda] numpy                     1.18.4                   pypi_0    pypi
[conda] numpydoc                  0.7.0            py36h18f165f_0  
[conda] torch                     1.5.0+cu101              pypi_0    pypi
[conda] torchvision               0.6.0+cu101              pypi_0    pypi


- first, check that your computer detect your graphic card ``GPU 0: ...``
- second, you need a driver to use your GPU for any application: ``Nvidia driver version: ...``
- third, you need some libraries to run any data science algorith on your GPU: ``cuDNN version: ...`` and ``CUDA runtime version: ...``
- fourth, you need to install Pytorch after having all the things above so that Pytorch is built using the CUDA library: ``PyTorch version: 1.5.0+cu101`` and ``CUDA used to build PyTorch: 10.1``
- Finally, if your are on windows, you might also need ``Microsoft Visual C++ 14.0``

If any of these are missing, the project setup will probably fail somewhere. Below are the instruction to fix your environment.

## Setup


In [ ]:
! apt-get update
! pip install --upgrade pip

! apt-get install linux-headers-$(uname -r)
! apt-get -y install cmake

! conda update conda
! conda update conda-build

In [ ]:
!gcc --version

gcc (Ubuntu 5.4.0-6ubuntu1~16.04.9) 5.4.0 20160609
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



## Driver



In [ ]:
# this command must work otherwise check the symLink /usr/local/cuda
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Fri_Feb__8_19:08:17_PST_2019
Cuda compilation tools, release 10.1, V10.1.105


## CUDA
Note that each version of CUDA has a minimum requirement concerning the version of the driver


cuda toolkit: https://developer.nvidia.com/cuda-10.1-download-archive-base?target_os=Linux&target_arch=x86_64&target_distro=Ubuntu&target_version=1604&target_type=runfilelocal
example of thing you should install XD
```
wget https://developer.nvidia.com/compute/cuda/10.1/Prod/local_installers/cuda_10.1.105_418.39_linux.run
sh cuda_10.1.105_418.39_linux.run
```

## libnccl
Nvidia NCCL can be downloaded from: https://developer.nvidia.com/nccl/nccl-download (you need to create a free account)

## libcudnn
do not download ``libcudnn-dev``

then install it using this command ``dpkg -i libcudnn7_7.6.5.32-1+cuda10.1_amd64.deb``


## Pytorch
Get the command line that fits your hardware on this web site: https://pytorch.org/get-started/locally/
For example you should run something like the line below

In [ ]:
! pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
# now run the command below to see if Pytorch detects your GPUs
import torch
torch.cuda.is_available()

True

# Install the requirements
[github source](https://github.com/KaihuaTang/Scene-Graph-Benchmark.pytorch)

## Python libraries
Might take 5 to 10 mins to install all the rest.

In [ ]:
! pip install ipython scipy h5py

In [ ]:
! pip install ninja yacs cython matplotlib tqdm opencv-python overrides

ends with `Successfully installed ninja-1.9.0.post1 overrides-3.0.0 yacs-0.1.7`

## install PyCOCO tools (cocoapi)

In [ ]:
! git clone https://github.com/cocodataset/cocoapi.git

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 3.13 MiB/s, done.
Resolving deltas: 100% (576/576), done.
Checking connectivity... done.


In [ ]:
! cd cocoapi/PythonAPI; python setup.py build_ext install

ends with

```
Finished processing dependencies for pycocotools==2.0
```



## install apex
it is a PyTorch extension for easy mixed precision and distributed training in Pytorch

### method 1

In [ ]:
! git clone https://github.com/NVIDIA/apex.git
! cd apex ; python setup.py install --cuda_ext --cpp_ext

### method 2
https://stackoverflow.com/questions/57284345/how-to-install-nvidia-apex-on-google-colab

In [ ]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setup.sh


In [ ]:
!sh setup.sh

In [ ]:
!pip install -v --no-cache-dir ./apex

ends with
```
Successfully installed apex-0.1
Cleaning up...
Removed build tracker '/tmp/pip-req-tracker-yl9p9317'
```

"A Python-only build" omits:
* Fused kernels **required** to use apex.optimizers.FusedAdam.
* Fused kernels **required** to use apex.normalization.FusedLayerNorm.
* Fused kernels that improve the performance and numerical stability of apex.parallel.SyncBatchNorm.
* Fused kernels that improve the performance of apex.parallel.DistributedDataParallel and apex.amp. DistributedDataParallel, amp, and SyncBatchNorm will still be usable, but they may be slower.

AMP: Automatic Mixed Precision: https://nvidia.github.io/apex/amp.html

## install PyTorch Detection (Scene-Graph-Benchmark.pytorch)
We change the file name from `Scene-Graph-Benchmark.pytorch` to `Scene`
because *ninja* can not handel certain characters in the directory's name
([source](https://stackoverflow.com/questions/54569963/error-building-depfile-has-multiple-output-paths-ninja-build-stopped-subcomm ))

In [ ]:
! git clone https://github.com/KaihuaTang/Scene-Graph-Benchmark.pytorch.git Scene

Cloning into 'Scene-Graph-Benchmark.pytorch'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 462 (delta 1), reused 0 (delta 0), pack-reused 451
Receiving objects: 100% (462/462), 26.17 MiB | 12.90 MiB/s, done.
Resolving deltas: 100% (150/150), done.
Checking connectivity... done.


In [ ]:
# you can also rename it using this cmd
# ! mv Scene-Graph-Benchmark.pytorch Scene

In [ ]:
! cd Scene; python setup.py build develop

running build
running build_py
running build_ext
building 'maskrcnn_benchmark._C' extension
Emitting ninja build file /root/Scene/build/temp.linux-x86_64-3.7/build.ninja...
Compiling objects...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
[1/2] /usr/local/cuda/bin/nvcc -DWITH_CUDA -I/root/Scene/maskrcnn_benchmark/csrc -I/opt/conda/lib/python3.7/site-packages/torch/include -I/opt/conda/lib/python3.7/site-packages/torch/include/torch/csrc/api/include -I/opt/conda/lib/python3.7/site-packages/torch/include/TH -I/opt/conda/lib/python3.7/site-packages/torch/include/THC -I/usr/local/cuda/include -I/opt/conda/include/python3.7m -c -c /root/Scene/maskrcnn_benchmark/csrc/cuda/deform_pool_cuda.cu -o /root/Scene/build/temp.linux-x86_64-3.7/root/Scene/maskrcnn_benchmark/csrc/cuda/deform_pool_cuda.o -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-relaxed-constexpr --compiler-options 

ends with:


```
Installed /content/Scene
Processing dependencies for maskrcnn-benchmark==0.1
Finished processing dependencies for maskrcnn-benchmark==0.1
```
otherwise you can get:
~~~
RuntimeError: Error compiling objects for extension
~~~


The line above might not work for many reasons:
* ninja is not installed
* the folder name contains crasy characters like space, points
* other reasons

follow carefully the instructions above to avoid any problem :)

### Additionnal instructions for Windows:
You might incounter this issue https://github.com/facebookresearch/maskrcnn-benchmark/issues/547
then you might need to run that first:
```
set "VS150COMNTOOLS=C:\Program Files (x86)\Microsoft Visual Studio\2019\Community\VC\Auxiliary\Build"
set CMAKE_GENERATOR=Visual Studio 16 2019 Win64
set DISTUTILS_USE_SDK=1
call "%VS150COMNTOOLS%\vcvarsall.bat" x64 -vcvars_ver=14.11
python setup.py build develop
call "C:\Program Files (x86)\Microsoft Visual Studio 14.0\VC\vcvarsall.bat"  x64 -vcvars_ver=14.0
```

# DATASET
## VG images
Download the VG images part1 (9 Gb) part2 (5 Gb).

In [ ]:
! wget https://cs.stanford.edu/people/rak248/VG_100K_2/images.zip

In [ ]:
! wget https://cs.stanford.edu/people/rak248/VG_100K_2/images2.zip

--2020-06-04 13:58:39--  https://cs.stanford.edu/people/rak248/VG_100K_2/images2.zip
Resolving cs.stanford.edu (cs.stanford.edu)... 171.64.64.64
Connecting to cs.stanford.edu (cs.stanford.edu)|171.64.64.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5471658058 (5.1G) [application/zip]
Saving to: ‘images2.zip’

images2.zip         100%[===================>]   5.10G  4.98MB/s    in 30m 18s 

2020-06-04 14:28:57 (2.87 MB/s) - ‘images2.zip’ saved [5471658058/5471658058]



Please Extract these images to the directory datasets/vg/VG_100K/

If you want to use other directory, please link it in `DATASETS['VG_stanford_filtered']['img_dir']` of `maskrcnn_benchmark/config/paths_catelog.py`

## SGG model

The following code does this automatically:

Download the [scene graphs](https://onedrive.live.com/embed?cid=22376FFAD72C4B64&resid=22376FFAD72C4B64%21779871&authkey=AA33n7BRpB1xa3I) to `Scene/datasets/vg/VG-SGG-with-attri.h5` (144 Mb)

or you can edit the path in `DATASETS['VG_stanford_filtered_with_attribute']['roidb_file']` of `maskrcnn_benchmark/config/paths_catelog.py`

In [ ]:
!pip install gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-3.11.0-py3-none-any.whl size=9619 sha256=184ea18d44683eac553a1b7705ea72a9ec3cf61445acecbfd08fdcf3912a97bb
  Stored in directory: /root/.cache/pip/wheels/05/e6/10/9cbfea8dcf9fde0f406da1e4c71d5c3cf3c99e0502d7f08ac6
Successfully built gdown


In [ ]:
# alternatively
! gdown "https://drive.google.com/uc?id=1h2XzeQgJNYgg3q66t1oujofbWvBIYMXG" -O /content/Scene/maskrcnn_benchmark/data/datasets/vg/VG-SGG-with-attri.h5
# miror link for manual download https://onedrive.live.com/embed?cid=22376FFAD72C4B64&resid=22376FFAD72C4B64%21779871&authkey=AA33n7BRpB1xa3I


In [ ]:
! ls -l --block-size=M "/content/Scene/maskrcnn_benchmark/datasets/vg"
# file size should be 144M

## Pretrained models

do not name any directory `checkpoints` with an ``s`` because you will not be able to explore it using jupyter notebook

In [ ]:
%%bash
mkdir Scene/checkpoint/pretrained_faster_rcnn/
cd Scene/checkpoint/pretrained_faster_rcnn/
gdown "https://drive.google.com/uc?id=1GoUdVlwZ8ekS7w_aWJ-tcXsx-ULCCjyI" -O log.txt
gdown "https://drive.google.com/uc?id=1Pj8gfFBouqaKzJVkOV6wsY8GU60z6Nrb" -O config.yml
gdown "https://drive.google.com/uc?id=1TRT3uX0tbqvIfNeL3bRGzVeqKS8SHtFa" -O model_final.pth
gdown "https://drive.google.com/uc?id=1Y1SnKGeQCBqGmIpUa8izy2EvUYLyPc89" -O VG_stanford_filtered_wth_attribute_train_statistics.cache
gdown "https://drive.google.com/uc?id=1_aRGThcciCvg0gFLr9EkhLIE92vEitfP" -O labels.json
gdown "https://drive.google.com/uc?id=1q6w_tZzhKTx70hgmQ-7Rnlt4EM60MmXp" -O last_checkpoint

If the links above are dead you can download and extract the files manually following this:

- [download the Faster R-CNN model](https://onedrive.live.com/embed?cid=22376FFAD72C4B64&resid=22376FFAD72C4B64%21779870&authkey=AH5CPVb9g5E67iQ),
- extract all the files to the directory `/home/username/checkpoints/pretrained_faster_rcnn`.

To train your own Faster R-CNN model, please follow [the next section](https://github.com/KaihuaTang/Scene-Graph-Benchmark.pytorch#pretrained-models).


# Setup done !
# Training and Testing
## Settings
The default settings are under

`configs/e2e_relation_X_101_32_8_FPN_1x.yaml` ([see on github](https://github.com/KaihuaTang/Scene-Graph-Benchmark.pytorch/blob/db02790a60bb9b9f7c270352820968b2f2089469/configs/e2e_relation_X_101_32_8_FPN_1x.yaml#L74))
and
`maskrcnn_benchmark/config/defaults.py` (todo find link in github)

The priority is in this order `command > yaml > defaults.py`

* For Predicate Classification (PredCls), we need to set:
```
MODEL.ROI_RELATION_HEAD.USE_GT_BOX True
MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL True
```

* For Unbiased-Causal-TDE Model:
```
MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor
```

## Example 1 : (PreCls, Motif Model)
### Training Example 1 : (PreCls, Motif Model)

`CUDA_VISIBLE_DEVICES=0,1` <-- will use GPU 0 and 1

`python -m torch.distributed.launch` <-- will run the script across multiple GPUs

`--master_port 10025` <-- the value itself is not important, just use a free port

`--nproc_per_node=2` <-- [{num_gpus}](https://docs.fast.ai/distributed.html): this should correspond to the number of gpu you specified up


`tools/relation_train_net.py` <-- the script to run by the "torch.distributed". Mainly we want to train the model, or resume the training
`--config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml"` <-- Config file for all what we didn't specify
`MODEL.ROI_RELATION_HEAD.USE_GT_BOX True ` <--
`MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL True ` <-- This means that the ground truth object labels are provided as input to the model.

`MODEL.ROI_RELATION_HEAD.PREDICTOR MotifPredictor ` <-- round truth bounding boxes are provided as input to the model.

`SOLVER.IMS_PER_BATCH 12` <--        **this number must be divisible by the number of GPUs (2) used.**

`TEST.IMS_PER_BATCH 2` <--            **must be equal to the number of GPUs (2) used.**

`DTYPE "float16" ` <--

`SOLVER.MAX_ITER 50000 ` <-- number of epoch, there is also EarlyStopping implemented

`SOLVER.VAL_PERIOD 2000 ` <-- run validation every 2 000 epochs

`SOLVER.CHECKPOINT_PERIOD 2000 ` <-- create a checkpoint every 2000 epochs (1 hour or more)

`GLOVE_DIR /home/kaihua/glove ` <-- directory where the pretrained word embeddings will be downloaded and stored

`MODEL.PRETRAINED_DETECTOR_CKPT /home/kaihua/checkpoint/pretrained_faster_rcnn/model_final.pth ` <--

`OUTPUT_DIR /home/kaihua/checkpoints/motif-precls-exmp` <-- where the model is saved. if the directory is not empty then the training is automatically resumed. So you can for example stop training and add more GPUs



In [ ]:
! cd "Scene/"; python -m torch.distributed.launch --master_port 10025 --nproc_per_node=8 \
tools/relation_train_net.py --config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" \
MODEL.ROI_RELATION_HEAD.USE_GT_BOX True \
MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL True \
MODEL.ROI_RELATION_HEAD.PREDICTOR MotifPredictor \
SOLVER.IMS_PER_BATCH 12 TEST.IMS_PER_BATCH 2 DTYPE "float16" \
SOLVER.MAX_ITER 50000 SOLVER.VAL_PERIOD 2000 SOLVER.CHECKPOINT_PERIOD 2000 \
GLOVE_DIR glove/ \
MODEL.PRETRAINED_DETECTOR_CKPT checkpoint/pretrained_faster_rcnn/model_final.pth \
OUTPUT_DIR checkpoint/motif-precls-exmp

### Test

### Test Example 1 : (PreCls, Motif Model)
Better use only one GPU for testing
~~~
CUDA_VISIBLE_DEVICES=0
python -m torch.distributed.launch
--master_port 10027
--nproc_per_node=1
~~~

`tools/relation_test_net.py` <-- this line is the only that change

~~~
--config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml"
MODEL.ROI_RELATION_HEAD.USE_GT_BOX True
MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL True
MODEL.ROI_RELATION_HEAD.PREDICTOR MotifPredictor
~~~
`TEST.IMS_PER_BATCH 1` <----------------------------- must be equal to nproc_per_node
~~~
DTYPE "float16"
GLOVE_DIR /home/kaihua/glove
MODEL.PRETRAINED_DETECTOR_CKPT checkpoints/motif-precls-exmp
OUTPUT_DIR checkpoints/motif-precls-exmp
~~~

In [ ]:
# evaluation
! cd "Scene/"; python -m torch.distributed.launch --master_port 10027 --nproc_per_node=8 tools/relation_test_net.py \
--config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" \
MODEL.ROI_RELATION_HEAD.USE_GT_BOX True \
MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL True \
MODEL.ROI_RELATION_HEAD.PREDICTOR MotifPredictor \
TEST.IMS_PER_BATCH 32 DTYPE "float16" \
GLOVE_DIR glove/ \
MODEL.PRETRAINED_DETECTOR_CKPT checkpoint/pretrained_faster_rcnn/model_final.pth \
OUTPUT_DIR checkpoint/motif-precls-exmp

## Train and test Example 2 : (SGCls, Causal, TDE, SUM Fusion, MOTIFS Model)

In [ ]:
!  python -m torch.distributed.launch --master_port 10026 --nproc_per_node=2 \
tools/relation_train_net.py --config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" \
MODEL.ROI_RELATION_HEAD.USE_GT_BOX True \
MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False \
MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor \
MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE none
MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum \
MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs \
SOLVER.IMS_PER_BATCH 12 TEST.IMS_PER_BATCH 2 DTYPE "float16" \
SOLVER.MAX_ITER 50000 SOLVER.VAL_PERIOD 2000 SOLVER.CHECKPOINT_PERIOD 2000 \
GLOVE_DIR /home/kaihua/glove
MODEL.PRETRAINED_DETECTOR_CKPT /home/kaihua/checkpoints/pretrained_faster_rcnn/model_final.pth
OUTPUT_DIR /home/kaihua/checkpoints/causal-motifs-sgcls-exmp

In [ ]:
# evaluation
CUDA_VISIBLE_DEVICES=0 python -m torch.distributed.launch --master_port 10028 --nproc_per_node=1 tools/relation_test_net.py --config-file "configs/e2e_relation_X_101_32_8_FPN_1x.yaml" MODEL.ROI_RELATION_HEAD.USE_GT_BOX True MODEL.ROI_RELATION_HEAD.USE_GT_OBJECT_LABEL False MODEL.ROI_RELATION_HEAD.PREDICTOR CausalAnalysisPredictor MODEL.ROI_RELATION_HEAD.CAUSAL.EFFECT_TYPE TDE MODEL.ROI_RELATION_HEAD.CAUSAL.FUSION_TYPE sum MODEL.ROI_RELATION_HEAD.CAUSAL.CONTEXT_LAYER motifs  TEST.IMS_PER_BATCH 1 DTYPE "float16" GLOVE_DIR /home/kaihua/glove MODEL.PRETRAINED_DETECTOR_CKPT /home/kaihua/checkpoints/causal-motifs-sgcls-exmp OUTPUT_DIR /home/kaihua/checkpoints/causal-motifs-sgcls-exmp